# **Lab 4: Machine Learning Library (MLlib)**

Lecturer: `Sirasit Lochanachit`


Course: `06026213 Big Data Systems`

Term: `02/2024`

Lab materials prepared by `Duangkamon Phobsungnoen (TA)`

# **Example 1: Estimator, Transformer, and Param**

**STEP 0 : ติดตั้ง PySpark**



In [1]:
!pip install pyspark

**STEP 1 : สร้าง SparkSession**

In [34]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("PySparkExample").getOrCreate()

ใช้บน local ของ colab & ใช้แค่ 1 thread ใน cpu = ไม่ได้เกิดการประมวลผลแบบ parallel

**STEP 2 : สร้าง DataFrame**

สร้าง DataFrame สำหรับฝึกโมเดลที่ประกอบด้วย list ของ (label, features) tuples

**Feature Vectors**
- ข้อมูลที่ใช้ train ของ MLlib ควรอยู่ในรูปของ feature vectors
- ข้อมูลของแต่ละแถวจะอยู่ใน vector (list) เดียวกัน โดยสมาชิกของ vector คือค่าของแต่ละ column


**Label index**
- Column ที่เป็น label ต้องมีค่าเป็น 0/1 ไม่สามารถเป็น string เช่น Yes/No ได้


In [35]:
from pyspark.ml.linalg import Vectors

#ตัวแรก คือ target label, ต่อมาคือ vector, each row in the same list
training = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])
# df นี้มี 2 col, 1 vector = 1 col แล้ว

**เพิ่มเติม:** Vectors.dense คือวิธีการสร้างเวกเตอร์ที่มีค่าคงที่ (dense vector) ซึ่งถูกใช้เพื่อแทนข้อมูลในรูปแบบของเวกเตอร์ในลักษณะทางคณิตศาสตร์ เช่น การเก็บข้อมูลหลายมิติที่มีค่าเป็นตัวเลขในรูปแบบของเวกเตอร์

ตัวอย่าง

`Vectors.dense([0.0, 1.1, 0.1])`

จะสร้างเวกเตอร์ที่มีสามค่า คือ 0.0, 1.1, และ 0.1 ซึ่งแต่ละค่าจะเป็นข้อมูลที่ใช้แทนคุณสมบัติต่างๆ ในการเรียนรู้ของโมเดล machine learning เช่น classification หรือ regression

**STEP 3 : สร้าง Logistic Regression Estimator**

In [36]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=10)

แสดง Paramater ทั้งหมด รวมถึงคำอธิบายประกอบและค่า default

In [37]:
print("LogisticRegression parameters:\n" + lr.explainParams() + "\n")

LogisticRegression parameters:
aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The bou

**STEP 4 : ฝึกโมเดลและดู parameter**

In [38]:
model1 = lr.fit(training)

In [39]:
print("Model 1 was fit using parameters: ")
print(model1.extractParamMap())

Model 1 was fit using parameters: 
{Param(parent='LogisticRegression_5d383996d31c', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2, Param(parent='LogisticRegression_5d383996d31c', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0, Param(parent='LogisticRegression_5d383996d31c', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'auto', Param(parent='LogisticRegression_5d383996d31c', name='featuresCol', doc='features column name.'): 'features', Param(parent='LogisticRegression_5d383996d31c', name='fitIntercept', doc='whether to fit an intercept term.'): True, Param(parent='LogisticRegression_5d383996d31c', name='labelCol', doc='label column name.'): 'label', Param(parent='LogisticRegression_5d383996d31c', name='maxBlockSizeInMB', do

**STEP 5 : ปรับแต่ง parameter ด้วย ParamMap**

สามารถเปลี่ยนแปลงพารามิเตอร์ต่างๆ เช่น maxIter, regParam เป็นต้น และใช้ ParamMap ในการกำหนด parameter

In [40]:
# สร้าง param map เป็น dict ของ parameter config
paramMap = {lr.maxIter: 20, lr.regParam: 0.1}

# update param
paramMap[lr.maxIter] = 30 # method 1 : index
paramMap.update({lr.threshold: 0.55}) # method 2 : update

# paramMap
model2 = lr.fit(training, paramMap)

In [41]:
print("Model 2 was fit using parameters: ")
print(model2.extractParamMap())

Model 2 was fit using parameters: 
{Param(parent='LogisticRegression_5d383996d31c', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2, Param(parent='LogisticRegression_5d383996d31c', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0, Param(parent='LogisticRegression_5d383996d31c', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'auto', Param(parent='LogisticRegression_5d383996d31c', name='featuresCol', doc='features column name.'): 'features', Param(parent='LogisticRegression_5d383996d31c', name='fitIntercept', doc='whether to fit an intercept term.'): True, Param(parent='LogisticRegression_5d383996d31c', name='labelCol', doc='label column name.'): 'label', Param(parent='LogisticRegression_5d383996d31c', name='maxBlockSizeInMB', do

**STEP 6 : สร้างข้อมูลทดสอบ (test data)**

In [42]:
test = spark.createDataFrame([
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["label", "features"])

**STEP 7 : ทำนายผลจากโมเดล**

ใช้ Transformer เพื่อทำการทำนายผลจากข้อมูลทดสอบ

In [43]:
prediction = model2.transform(test) # แปลง test df ให้เป็นอีก df นึง
prediction

DataFrame[label: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [44]:
result = prediction.select('features', 'label', 'probability', 'prediction').collect()
for row in result:
  print('feature = %s, label = %s -> prob = %s, prediction = %s' % (row.features, row.label, row.probability, row.prediction))

feature = [-1.0,1.5,1.3], label = 1.0 -> prob = [0.05707304993572542,0.9429269500642746], prediction = 1.0
feature = [3.0,2.0,-0.1], label = 0.0 -> prob = [0.9238521956443227,0.07614780435567725], prediction = 0.0
feature = [0.0,2.2,-1.5], label = 1.0 -> prob = [0.10972780286187782,0.8902721971381222], prediction = 1.0


In [45]:
model_test_result = model2.evaluate(test)

model_test_result

In [48]:
model_test_result.accuracy, model_test_result.recallByLabel

(1.0, [1.0, 1.0])

# **Example 2: การทำ Pipelines ในการฝึกและทำนายโมเดล Logistic Regression**

## Ex 2.1: ข้อมูลไม่ได้อยู่ในรูปของ Vector

**STEP 1 : ใช้ Pipeline ในการสร้างกระบวนการ**

สร้าง Pipeline ที่ประกอบด้วยหลายๆ ขั้นตอน เช่น Tokenizer, HashingTF และ LogisticRegression

In [14]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF

# เตรียมข้อมูลฝึกจำนวน 3 column
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

Pipeline - Train Model

In [15]:
# กำหนด stage in pipeline
tokenizer = Tokenizer(inputCol = "text", outputCol = "words")
hashingTF = HashingTF(inputCol = tokenizer.getOutputCol(), outputCol = "features")
# tokenizer.getOutputCol() คำสั่งนี้ safe หน่อย
lr = LogisticRegression(maxIter = 10, regParam = 0.001)

# Pipeline
pipeline = Pipeline(stages=[tokenizer,hashingTF, lr])

# Pipeline Training (ฝึก model)
model = pipeline.fit(training) # pipeline เป็น estimator

In [16]:
type(model)

pyspark.ml.pipeline.PipelineModel

**STEP 2 : ทำนายผลด้วย Pipeline**

In [17]:
# เตรียมข้อมูล test ประกอบด้วย 2 column
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])


Pipeline - Test Model

In [18]:
prediction = model.transform(test) # เรียก pipeline บน test data
selected = prediction.select('id', 'text', 'probability', 'prediction')
for row in selected.collect():
  print('(%d, %s) --> prob = %s, prediction = %f' % (row.id, row.text, row.probability, row.prediction))

(4, spark i j k) --> prob = [0.6292098489668483,0.3707901510331517], prediction = 0.000000
(5, l m n) --> prob = [0.984770006762304,0.015229993237696027], prediction = 0.000000
(6, spark hadoop spark) --> prob = [0.13412348342566147,0.8658765165743385], prediction = 1.000000
(7, apache hadoop) --> prob = [0.9955732114398529,0.00442678856014711], prediction = 0.000000


## Ex 2.2: แปลงข้อมูลอยู่ในรูปของ Vector

ข้อมูลตัวอย่างประกอบด้วย column ต่างๆ เช่น
- label
- category (ประเภท categorical)
- feature1, feature2 (คุณสมบัติหรือฟีเจอร์เชิงตัวเลข)

**STEP 1 : สร้าง Spark Session**

In [52]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PipelineExample").getOrCreate()

**STEP 2 : สร้าง DataFrame สำหรับการฝึกโมเดล**

In [53]:
data = spark.createDataFrame([
    (0, "a", 1.0, 2.0),
    (1, "b", 2.0, 1.0),
    (0, "a", 2.0, 3.0),
    (1, "b", 1.0, 2.0),
], ["label", "category", "feature1", "feature2"])

**STEP 3 : การแปลงข้อมูล - StringIndexer และ VectorAssembler**

- StringIndexer: ใช้ในการแปลงค่าที่เป็นประเภท categorical (เช่น "a", "b") ให้เป็นตัวเลข

- VectorAssembler: ใช้ในการรวมฟีเจอร์หลายๆ คอลัมน์ให้เป็นคอลัมน์เดียวที่สามารถนำไปใช้ฝึกโมเดลได้

In [54]:
from pyspark.ml.feature import StringIndexer, VectorAssembler

indexer = StringIndexer(inputCol="category", outputCol='categoryIndex',
                        stringOrderType='frequencyDesc')
assemble = VectorAssembler(inputCols=["feature1", "feature2", "categoryIndex"],
                           outputCol="features")

**STEP 4 : การเลือกโมเดล - Logistic Regression**

ในขั้นตอนนี้ เราจะเลือกโมเดล Logistic Regression เพื่อใช้ในการฝึกโมเดลสำหรับการจำแนกประเภท

In [55]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="features", labelCol="label",
                        maxIter=1000, tol=0.0001, regParam=0.01)

**STEP 5 : การสร้าง Pipeline**

ในขั้นตอนนี้ เราจะรวมขั้นตอนต่างๆ ไว้ใน Pipeline ซึ่งประกอบด้วย

- การแปลงข้อมูลด้วย StringIndexer
- การรวมฟีเจอร์ด้วย VectorAssembler
- การฝึกโมเดลด้วย LogisticRegression

In [56]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[indexer, assemble, lr])

**STEP 6 : การฝึกโมเดล**

หลังจากสร้าง Pipeline เสร็จแล้ว เราจะใช้ fit() เพื่อฝึกโมเดลด้วยข้อมูลที่เตรียมไว้

In [57]:
model = pipeline.fit(data)

**STEP 7 : การทำนายผล (Prediction)**

In [58]:
predictions = model.transform(data)

predictions.show()

+-----+--------+--------+--------+-------------+-------------+--------------------+--------------------+----------+
|label|category|feature1|feature2|categoryIndex|     features|       rawPrediction|         probability|prediction|
+-----+--------+--------+--------+-------------+-------------+--------------------+--------------------+----------+
|    0|       a|     1.0|     2.0|          0.0|[1.0,2.0,0.0]|[2.79474715118196...|[0.94239130912281...|       0.0|
|    1|       b|     2.0|     1.0|          1.0|[2.0,1.0,1.0]|[-4.0629416230743...|[0.01690757117189...|       1.0|
|    0|       a|     2.0|     3.0|          0.0|[2.0,3.0,0.0]|[4.06294162307433...|[0.98309242882810...|       0.0|
|    1|       b|     1.0|     2.0|          1.0|[1.0,2.0,1.0]|[-2.7947471511819...|[0.05760869087718...|       1.0|
+-----+--------+--------+--------+-------------+-------------+--------------------+--------------------+----------+



Model Description

In [ ]:
model.coefficients

In [ ]:
model.intercept

### Model Evaluation

In [62]:
model_test_result = model.evaluate(test)

model_test_result

AttributeError: 'PipelineModel' object has no attribute 'evaluate'

# **Extracting, transforming and selecting features**

Algorithm ต่างๆ ที่ใช้ในการทำงานกับ feature สามารถแบ่งออกเป็นกลุ่มต่างๆ ดังนี้

- **Extraction**
- **Transformation**
- **Selection**


### **Feature Extractors**

Feature Extractors คือ กระบวนการที่ใช้ในการดึงเอาคุณสมบัติ (features) ที่สำคัญจากข้อมูลในรูปแบบดิบ (raw data) เพื่อนำมาประมวลผลและใช้ในการฝึกโมเดล Machine Learning. การใช้ Feature Extractors ช่วยให้สามารถคัดเลือกข้อมูลที่มีความหมายและเป็นประโยชน์สำหรับการทำนายหรือการจำแนกประเภทได้ดีขึ้น ลดความซับซ้อนของข้อมูลและเพิ่มประสิทธิภาพในการเรียนรู้ของโมเดล

**1. TF-IDF (Term Frequency-Inverse Document Frequency)**


เป็นวิธีการแปลงข้อมูลจากข้อความให้เป็นเวกเตอร์ที่สะท้อนความสำคัญของคำในเอกสาร โดยวิธีนี้จะช่วยให้สามารถแยกแยะคำที่สำคัญและไม่สำคัญในเอกสารนั้นได้

  หลักการของ TF-IDF ประกอบไปด้วย 2 ส่วนหลัก
  
    1. Term Frequency (TF): คือจำนวนครั้งที่คำ t ปรากฏในเอกสาร d
    2. Inverse Document Frequency (IDF): คือค่าที่วัดความสำคัญของคำ t ในฐานข้อมูล (corpus) โดยจะคำนวณจากจำนวนเอกสารที่คำนี้ปรากฏ (DF) ในฐานข้อมูลทั้งหมด D

In [63]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("TF-IDF Example").getOrCreate()

sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

tokenizer = Tokenizer(inputCol='sentence', outputCol="words")
hashTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="rawFeatures", numFeatures=20)
idf = IDF(inputCol=hashTF.getOutputCol(), outputCol="features")

In [64]:
pipeline = Pipeline(stages=[tokenizer, hashTF, idf])
model = pipeline.fit(sentenceData)
predictions = model.transform(sentenceData)

In [66]:
predictions.select('label', 'features').show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(20,[6,8,13,16],[...|
|  0.0|(20,[0,2,7,13,15,...|
|  1.0|(20,[3,4,6,11,19]...|
+-----+--------------------+



**2. Word2Vec**

เป็น Estimator ที่ใช้สำหรับแปลงคำแต่ละคำในเอกสารให้เป็นเวกเตอร์ขนาดคงที่ โดยการเรียนรู้จากชุดข้อมูลที่เป็นประโยคหรือเอกสารที่มีการแยกคำ (bag of words) ซึ่งจะช่วยในการแปลงคำในเอกสารให้เป็นเวกเตอร์เพื่อใช้ในงานต่างๆ เช่น การคำนวณความคล้ายคลึงของเอกสาร หรือใช้เป็นคุณลักษณะ (feature) สำหรับการพยากรณ์

In [67]:
from pyspark.ml.feature import Word2Vec

documentDF = spark.createDataFrame([
    ("Hi I heard about Spark".split(" "), ),
    ("I wish Java could use case classes".split(" "), ),
    ("Logistic regression models are neat".split(" "), )
], ["text"])

word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text", outputCol="result") # Estimator
model = word2Vec.fit(documentDF)
result = model.transform(documentDF)

for row in result.collect():
  text, vector = row
  print("text: %s -> \nVector: %s \n" %(", ".join(text), vector))

text: Hi, I, heard, about, Spark -> 
Vector: [-0.026580248773097993,-0.02676662188023329,0.04222403690218926] 

text: I, wish, Java, could, use, case, classes -> 
Vector: [-0.034757427397250594,-0.05261531685079847,-0.0004108633313860212] 

text: Logistic, regression, models, are, neat -> 
Vector: [0.013996277935802937,0.04871041178703309,-0.03697966188192368] 



**3. CountVectorizer**

คือเครื่องมือที่ใช้แปลงข้อความเป็นเวกเตอร์ที่แสดงจำนวนการปรากฏของคำ (token counts) ในแต่ละเอกสาร โดยไม่จำเป็นต้องมีพจนานุกรมล่วงหน้า (a-priori dictionary) ด้วยการใช้ Estimator ในการสร้างพจนานุกรมจากข้อมูลที่มีอยู่ โมเดล CountVectorizerModel ที่ได้จากการฝึก (fitting) จะสร้างการแทนเอกสารในรูปแบบเวกเตอร์ที่แสดงการปรากฏของคำในพจนานุกรมที่ถูกสร้างขึ้น

  ในระหว่างการฝึกโมเดล, CountVectorizer จะเลือกคำที่มีความถี่สูงที่สุดในข้อมูล และกำหนดพารามิเตอร์ต่างๆ เช่น

  - vocabSize: ขนาดของพจนานุกรมที่ใช้

  - minDF: กำหนดจำนวนเอกสารขั้นต่ำที่คำต้องปรากฏถึงจะถูกใช้ในพจนานุกรม
  
  - binary: หากตั้งค่าเป็น true จะใช้เวกเตอร์แบบทวิภาค (binary vector) ซึ่งจะตั้งค่าที่ไม่เป็นศูนย์ทั้งหมดเป็น 1 (เหมาะกับโมเดลที่ทำงานกับข้อมูลแบบทวิภาค)

In [ ]:
from pyspark.ml.feature import CountVectorizer

df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

**4. FeatureHasher**

เป็นเทคนิคการแปลงข้อมูลที่ใช้การแฮชเพื่อทำให้ชุดของฟีเจอร์ที่มีลักษณะทั้งประเภทหมวดหมู่ (categorical) และตัวเลข (numerical) ถูกแปลงเป็นเวกเตอร์ฟีเจอร์ในมิติที่กำหนด (โดยปกติจะมีขนาดเล็กกว่าพื้นที่ฟีเจอร์เดิมมาก) โดยใช้ "hashing trick" ในการจับคู่ฟีเจอร์กับตำแหน่งในเวกเตอร์

  ลักษณะการทำงานของ FeatureHasher กับคอลัมน์ประเภทต่างๆ ดังนี้

  - **Numeric columns** จะใช้ค่าแฮชของชื่อคอลัมน์เพื่อจับคู่ค่าฟีเจอร์กับตำแหน่งในเวกเตอร์ฟีเจอร์ โดยปกติแล้วฟีเจอร์ตัวเลขจะไม่ถูกมองเป็นฟีเจอร์หมวดหมู่ (ถึงแม้ว่าจะเป็นจำนวนเต็ม) แต่สามารถระบุให้เป็นฟีเจอร์หมวดหมู่ได้ด้วยการกำหนดคอลัมน์ที่ต้องการในพารามิเตอร์ categoricalCols

  - **String columns** ฟีเจอร์ประเภทนี้จะใช้ค่าของการแฮชจากการรวมชื่อคอลัมน์และค่าของคอลัมน์ (เช่น "column_name=value") เพื่อจับคู่กับตำแหน่งในเวกเตอร์ โดยใช้ค่าตัวชี้ (indicator value) เป็น 1.0 คล้ายกับการทำ one-hot encoding

  - **Boolean columns** ฟีเจอร์ประเภทนี้จะทำงานเหมือนกับคอลัมน์สตริง โดยใช้ค่าแฮชจาก "column_name=true" หรือ "column_name=false" กับค่าตัวชี้ 1.0

  - **ค่าที่หายไป (null)** ค่าที่หายไปจะถูกละเว้นในการแปลง (จะถือเป็นค่า 0 ในเวกเตอร์ฟีเจอร์)

In [ ]:
from pyspark.ml.feature import FeatureHasher

df = spark.createDataFrame([
    (2.2, True, "1", "foo"),
    (3.3, False, "2", "bar"),
    (4.4, False, "3", "baz"),
    (5.5, False, "4", "foo")
], ["real", "bool", "stringNum", "string"])

### **Feature Transformers**

**Feature Transformers** คือ กระบวนการที่ใช้ในการแปลงหรือเปลี่ยนแปลงคุณสมบัติ (features) ของข้อมูล เพื่อให้เหมาะสมกับการนำไปใช้ในโมเดล Machine Learning การแปลงนี้สามารถช่วยให้โมเดลเรียนรู้ข้อมูลได้ดีขึ้น หรือสามารถปรับข้อมูลให้เป็นมาตรฐานที่เหมาะสมกับอัลกอริธึมที่ใช้

ใน PySpark ฟังก์ชัน transform เป็นฟังก์ชันที่ใช้สำหรับทำการแปลงข้อมูล (transform data) หลังจากที่โมเดลหรือ Pipeline ถูกฝึกแล้ว (fit) เพื่อให้โมเดลสามารถนำข้อมูลใหม่ไปใช้ในการทำนายผลหรือการประมวลผลต่างๆ ได้

**เมื่อไหร่ที่เราจะใช้ transform?**

โดยปกติแล้ว หลังจากที่เราฝึกโมเดลหรือ Pipeline ด้วย fit() เสร็จแล้ว เราจะใช้ transform() เพื่อนำข้อมูลใหม่ (ที่อาจจะไม่เคยเห็นมาก่อน) ไปผ่านขั้นตอนที่ได้ทำการฝึกไว้แล้ว เช่น การแปลงข้อมูลและการทำนายผลลัพธ์

**1. Tokenizer & RegexTokenizer**

 ใช้ในการแยกคำจากข้อความ โดย Tokenizer จะใช้การแยกคำตามช่องว่าง ส่วน RegexTokenizer ใช้การแยกคำโดยใช้ regular expressions (regex) เพื่อให้สามารถปรับแต่งวิธีการแยกคำได้มากขึ้น

**Example**

In [ ]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

df = spark.createDataFrame([
    (0, "Hi I heard about Spark"),
    (1, "I wish Java could use case classes"),
    (2, "Logistic,regression,models,are,neat")
], ["id", "sentence"])

**2. StopWordsRemover**

ใช้ในการลบคำที่ไม่สำคัญ (stop words) เช่น "I", "the", "a" ออกจากชุดข้อมูลเพื่อลดความซับซ้อนและเพิ่มประสิทธิภาพ

**Example**

In [69]:
from pyspark.ml.feature import StopWordsRemover

df = spark.createDataFrame([
    (0, ["I", "saw", "the", "red", "balloon"]),
    (1, ["Mary", "had", "a", "little", "lamb"])
], ["id", "raw"])

remover = StopWordsRemover(inputCol="raw", outputCol="filtered")
remover.transform(df).show(truncate=False)

+---+----------------------------+--------------------+
|id |raw                         |filtered            |
+---+----------------------------+--------------------+
|0  |[I, saw, the, red, balloon] |[saw, red, balloon] |
|1  |[Mary, had, a, little, lamb]|[Mary, little, lamb]|
+---+----------------------------+--------------------+



**3. NGram**

ใช้ในการสร้างชุดคำ n-gram ซึ่งเป็นการจับคู่ของคำหลายๆ คำ (เช่น 2 คำ หรือ 3 คำ) เพื่อใช้ในงานที่ต้องการตรวจจับลำดับของคำ

**Example**

In [ ]:
from pyspark.ml.feature import NGram

df = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])

**4. Binarizer**

ใช้ในการแปลงข้อมูลเชิงตัวเลขให้กลายเป็นข้อมูลแบบไบนารี (0 หรือ 1) โดยอาศัยเกณฑ์ threshold ในการตัดสินใจ

**Example**

In [ ]:
from pyspark.ml.feature import Binarizer

df = spark.createDataFrame([
    (0, 0.1),
    (1, 0.8),
    (2, 0.2)
], ["id", "feature"])

**5. PCA (Principal Component Analysis)**

ใช้ในการลดมิติของข้อมูลจากหลายมิติให้เหลือน้อยลง โดยคำนวณเป็นองค์ประกอบหลัก (Principal Components)

**Example**

In [ ]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([(Vectors.sparse(5, [(1, 1.0), (3, 7.0)]),),
                            (Vectors.dense([2.0, 0.0, 3.0, 4.0, 5.0]),),
                            (Vectors.dense([4.0, 0.0, 0.0, 6.0, 7.0]),)], ["features"])

**6. PolynomialExpansion**

ใช้ในการขยายข้อมูลในรูปแบบของพหุนาม โดยการสร้างคุณลักษณะใหม่ๆ ที่เป็นการผสมผสานระหว่างคุณลักษณะเดิม

**Example**

In [ ]:
from pyspark.ml.feature import PolynomialExpansion
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([
    (Vectors.dense([2.0, 1.0]),),
    (Vectors.dense([0.0, 0.0]),),
    (Vectors.dense([3.0, -1.0]),)
], ["features"])


**7. Discrete Cosine Transform (DCT)**

ใช้ในการแปลงข้อมูลจากโดเมนเวลาไปยังโดเมนความถี่ เพื่อใช้ในงานประมวลผลสัญญาณ

**Example**

In [ ]:
from pyspark.ml.feature import DCT
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([
    (Vectors.dense([0.0, 1.0, -2.0, 3.0]),),
    (Vectors.dense([-1.0, 2.0, 4.0, -7.0]),),
    (Vectors.dense([14.0, -2.0, -5.0, 1.0]),)
], ["features"])

**8. StringIndexer**

ใช้ในการแปลงค่าของตัวแปรที่เป็นข้อความ (string) ให้เป็นตัวเลข ซึ่งมักใช้ในกรณีที่ข้อมูลเป็นหมวดหมู่ (categorical)

**Example**

In [ ]:
from pyspark.ml.feature import StringIndexer

df = spark.createDataFrame(
    [(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],
    ["id", "category"]
)

**9. IndexToString**

ใช้ในการแปลงค่าตัวเลขที่ได้จาก StringIndexer กลับเป็นข้อความเดิม

**Example**

In [ ]:
from pyspark.ml.feature import IndexToString, StringIndexer

df = spark.createDataFrame(
    [(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],
    ["id", "category"])

**10. OneHotEncoder**

ใช้ในการแปลงค่าหมวดหมู่ (categorical values) ให้เป็นเวกเตอร์ไบนารีที่มีค่า 1 สำหรับหมวดหมู่ที่เลือก และค่า 0 สำหรับหมวดหมู่อื่นๆ

**Example**

In [ ]:
from pyspark.ml.feature import OneHotEncoder

df = spark.createDataFrame([
    (0.0, 1.0),
    (1.0, 0.0),
    (2.0, 1.0),
    (0.0, 2.0),
    (0.0, 1.0),
    (2.0, 0.0)
], ["categoryIndex1", "categoryIndex2"])

**11. VectorIndexer**

ใช้ในการจัดการกับคุณลักษณะประเภทหมวดหมู่ในชุดข้อมูลที่อยู่ในรูปแบบเวกเตอร์ (Vector)

**Example**

In [ ]:
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([
    (0, Vectors.dense([1.0, 0.1, 0.2]),),
    (1, Vectors.dense([2.0, 1.1, 1.2]),),
    (2, Vectors.dense([3.0, 10.1, 10.2]),),
    (3, Vectors.dense([4.0, 100.1, 100.2]),)
], ["id", "features"])

**12. Interaction**

ใช้ในการคำนวณการมีปฏิสัมพันธ์ (interaction) ของคุณลักษณะหลายๆ ตัว โดยการสร้างเวกเตอร์ที่ประกอบไปด้วยผลคูณของค่าต่างๆ

**Example**

In [ ]:
from pyspark.ml.feature import Interaction, VectorAssembler

df = spark.createDataFrame(
    [(1, 1, 2, 3, 8, 4, 5),
     (2, 4, 3, 8, 7, 9, 8),
     (3, 6, 1, 9, 2, 3, 6),
     (4, 10, 8, 6, 9, 4, 5),
     (5, 9, 2, 7, 10, 7, 3),
     (6, 1, 1, 4, 2, 8, 4)],
    ["id1", "id2", "id3", "id4", "id5", "id6", "id7"])

**13. Normalizer**

ใช้ในการปรับขนาดเวกเตอร์ให้มีความยาวเท่ากับ 1 โดยใช้การทำ normalization ซึ่งช่วยปรับมาตรฐานของข้อมูลให้เหมาะสมกับการเรียนรู้ของเครื่อง

**Example**

In [ ]:
from pyspark.ml.feature import Normalizer
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([
    (0, Vectors.dense([1.0, 0.5, -1.0]),),
    (1, Vectors.dense([2.0, 1.0, 1.0]),),
    (2, Vectors.dense([4.0, 10.0, 2.0]),)
], ["id", "features"])

**14. StandardScaler**

ปรับขนาดข้อมูลโดยทำให้ค่าเฉลี่ยเป็นศูนย์และส่วนเบี่ยงเบนมาตรฐานเป็น 1 เหมาะสำหรับข้อมูลที่มีการกระจายเป็นปกติ โดยตั้งค่าหลัก withStd=True และ withMean=False

**Example**

In [70]:
from pyspark.ml.feature import StandardScaler
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([(0, Vectors.dense([1.0, 0.1, -1.0])),
 (1, Vectors.dense([2.0, 1.1, 1.0])),
  (2, Vectors.dense([3.0, 10.1, 3.0]))], ["id", "features"])

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(df)
scaledData = scalerModel.transform(df)
scaledData.select("id", "features", "scaledFeatures").show(truncate=False)

+---+--------------+-------------------------------+
|id |features      |scaledFeatures                 |
+---+--------------+-------------------------------+
|0  |[1.0,0.1,-1.0]|[1.0,0.018156825980064073,-0.5]|
|1  |[2.0,1.1,1.0] |[2.0,0.19972508578070483,0.5]  |
|2  |[3.0,10.1,3.0]|[3.0,1.8338394239864713,1.5]   |
+---+--------------+-------------------------------+



**15. RobustScaler**

ใช้ค่ามัธยฐานและช่วงควอนไทล์ (IQR) แทนค่าเฉลี่ยและส่วนเบี่ยงเบนมาตรฐาน ทำให้ทนทานต่อข้อมูลที่มีค่าผิดปกติ (outliers)

**Example**

In [ ]:
from pyspark.ml.feature import RobustScaler
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([(0, Vectors.dense([1.0, 0.1, -1.0])), (1, Vectors.dense([2.0, 1.1, 1.0])), (2, Vectors.dense([3.0, 10.1, 3.0]))], ["id", "features"])

**16. MinMaxScaler**

ปรับขนาดข้อมูลให้อยู่ในช่วงที่กำหนด (เช่น [0, 1]) โดยพิจารณาจากค่าต่ำสุดและค่าสูงสุดของแต่ละฟีเจอร์

**Example**

In [ ]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([
    (0, Vectors.dense([1.0, 0.1, -1.0]),),
    (1, Vectors.dense([2.0, 1.1, 1.0]),),
    (2, Vectors.dense([3.0, 10.1, 3.0]),)
], ["id", "features"])

**17. MaxAbsScaler**

ปรับขนาดข้อมูลในช่วง [-1, 1] โดยใช้ค่ามากสุดสัมบูรณ์ (maximum absolute value) ของแต่ละฟีเจอร์ และไม่ทำการศูนย์ค่าเฉลี่ย

**Example**

In [ ]:
from pyspark.ml.feature import MaxAbsScaler
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([
    (0, Vectors.dense([1.0, 0.1, -8.0]),),
    (1, Vectors.dense([2.0, 1.0, -4.0]),),
    (2, Vectors.dense([4.0, 10.0, 8.0]),)
], ["id", "features"])

**18. Bucketizer**

**Example**

แบ่งข้อมูลเป็นกลุ่ม (bins) ตามช่วงที่กำหนด ใช้ในการแบ่งข้อมูลต่อเนื่องออกเป็นช่วงๆ

In [ ]:
from pyspark.ml.feature import Bucketizer

splits = [-float("inf"), -0.5, 0.0, 0.5, float("inf")]

df = spark.createDataFrame([(-999.9,), (-0.5,), (-0.3,), (0.0,), (0.2,), (999.9,)], ["features"])

**19. ElementwiseProduct**

คูณข้อมูลในแต่ละฟีเจอร์ด้วยเวกเตอร์ที่กำหนด ซึ่งเรียกว่าการคูณเชิงองค์ประกอบ (element-wise multiplication)

**Example**

In [ ]:
from pyspark.ml.feature import ElementwiseProduct
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([(Vectors.dense([1.0, 2.0, 3.0]),), (Vectors.dense([4.0, 5.0, 6.0]),)], ["vector"])

**20. SQLTransformer**

ใช้ SQL ในการแปลงข้อมูล เช่น การเลือกและแปลงข้อมูลโดยใช้คำสั่ง SELECT

**Example**

In [ ]:
from pyspark.ml.feature import SQLTransformer

df = spark.createDataFrame([
    (0, 1.0, 3.0),
    (2, 2.0, 5.0)
], ["id", "v1", "v2"])

**21. VectorAssembler**

รวมหลายคอลัมน์เข้าด้วยกันเป็นเวกเตอร์เดียว ใช้ในขั้นตอนการเตรียมข้อมูลสำหรับการเรียนรู้ของเครื่อง

**Example**

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

dataset = spark.createDataFrame(
    [(0, 18, 1.0, Vectors.dense([0.0, 10.0, 0.5]), 1.0)],
    ["id", "hour", "mobile", "userFeatures", "clicked"])

**22. VectorSizeHint**

กำหนดขนาดของเวกเตอร์ในคอลัมน์ที่มีประเภทข้อมูล Vector เพื่อให้สามารถใช้งานใน Transformer อื่นๆ ที่ต้องการขนาดเวกเตอร์

**Example**

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import (VectorSizeHint, VectorAssembler)

dataset = spark.createDataFrame(
    [(0, 18, 1.0, Vectors.dense([0.0, 10.0, 0.5]), 1.0),
     (0, 18, 1.0, Vectors.dense([0.0, 10.0]), 0.0)],
    ["id", "hour", "mobile", "userFeatures", "clicked"])

**23. QuantileDiscretizer**

แบ่งข้อมูลต่อเนื่องออกเป็นกลุ่มตามควอนไทล์ เช่น การแบ่งข้อมูลออกเป็นหลายช่วงตามจำนวนถัง (buckets)

**Example**

In [ ]:
from pyspark.ml.feature import QuantileDiscretizer

df = spark.createDataFrame([(0, 18.0), (1, 19.0), (2, 8.0), (3, 5.0), (4, 2.2)], ["id", "hour"])

**24. Imputer**

เติมข้อมูลที่หายไป (missing values) โดยใช้ค่าเฉลี่ย (mean), ค่ามัธยฐาน (median), หรือค่ามากที่สุด (mode) ของข้อมูลที่มีอยู่

**Example**

In [ ]:
from pyspark.ml.feature import Imputer

df = spark.createDataFrame([ (1.0, float("nan")), (2.0, float("nan")), (float("nan"), 3.0), (4.0, 4.0), (5.0, 5.0) ], ["a", "b"])

### **Feature Selectors**

Feature Selectors คือกระบวนการในการเลือกฟีเจอร์ที่สำคัญและมีประโยชน์ที่สุดในการสร้างโมเดลการเรียนรู้ของเครื่อง (Machine Learning) จากชุดข้อมูลที่มีฟีเจอร์หลายตัว เพื่อให้โมเดลสามารถทำงานได้ดีขึ้น โดยการเลือกฟีเจอร์จะช่วยลดความซับซ้อนของโมเดล เพิ่มความเร็วในการฝึกโมเดล และลดโอกาสที่โมเดลจะเกิดการ Overfitting (การที่โมเดลเรียนรู้จากข้อมูลฝึกจนเกินไปจนไม่สามารถทำนายข้อมูลใหม่ได้ดี)

**1. VectorSlicer**

VectorSlicer เป็นเครื่องมือที่ใช้ในการแปลงข้อมูล (transformer) ซึ่งจะรับเวกเตอร์ฟีเจอร์และสร้างเวกเตอร์ฟีเจอร์ใหม่ที่มีส่วนย่อยของฟีเจอร์จากเวกเตอร์เดิม ใช้สำหรับการดึงฟีเจอร์จากคอลัมน์เวกเตอร์

- การเลือกฟีเจอร์ VectorSlicer รับคอลัมน์เวกเตอร์ที่มีดัชนีที่ระบุ แล้วส่งออกคอลัมน์เวกเตอร์ใหม่ที่มีค่าฟีเจอร์ที่เลือกจากดัชนีเหล่านั้น
- ประเภทของดัชนี
  - ดัชนีจำนวนเต็ม (Integer indices) ใช้ระบุตำแหน่งในเวกเตอร์ (ใช้ setIndices())
  - ดัชนีชื่อฟีเจอร์ (String indices): ใช้ระบุชื่อฟีเจอร์ในเวกเตอร์ (ใช้ setNames()) การใช้ดัชนีนี้จะต้องมี AttributeGroup ในคอลัมน์เวกเตอร์ เพราะการเลือกจะอิงตามชื่อฟีเจอร์
- การใช้ดัชนีทั้งสองประเภท สามารถใช้ดัชนีจำนวนเต็มและชื่อฟีเจอร์ร่วมกันได้ แต่จะต้องเลือกฟีเจอร์อย่างน้อยหนึ่งฟีเจอร์ และไม่สามารถมีการซ้ำซ้อนระหว่างดัชนีจำนวนเต็มและชื่อฟีเจอร์ได้
- ลำดับของฟีเจอร์ ผลลัพธ์จะเรียงลำดับฟีเจอร์จากดัชนีจำนวนเต็มก่อน (ตามลำดับที่กำหนด) แล้วตามด้วยฟีเจอร์ที่เลือกจากชื่อฟีเจอร์ (ตามลำดับที่กำหนด)
โดยรวมแล้ว VectorSlicer ช่วยให้สามารถเลือกและจัดระเบียบฟีเจอร์จากเวกเตอร์ได้ตามความต้องการ

**Example**

In [ ]:
from pyspark.ml.feature import VectorSlicer
from pyspark.ml.linalg import Vectors
from pyspark.sql.types import Row

df = spark.createDataFrame([
    Row(userFeatures=Vectors.sparse(3, {0: -2.0, 1: 2.3})),
    Row(userFeatures=Vectors.dense([-2.0, 2.3, 0.0]))])

**2. RFormula**

RFormula คือฟีเจอร์ที่ใช้ในการเลือกคอลัมน์ตามสูตรของโมเดล R โดยใช้ตัวดำเนินการบางประเภท เช่น ~, +, -, :, และ . โดยที่

`~` ใช้แยกเป้าหมายและตัวแปร

`+` รวมตัวแปร เช่น + 0 หมายถึงการลบค่า intercept

`-` ลบตัวแปร เช่น - 1 หมายถึงการลบ intercept

`:` ใช้สร้างปฏิสัมพันธ์ระหว่างตัวแปร

`.` ใช้เลือกคอลัมน์ทั้งหมดยกเว้นตัวแปรเป้าหมาย

RFormula จะสร้างคอลัมน์เวกเตอร์ของฟีเจอร์และคอลัมน์ของป้ายกำกับที่เป็นตัวเลขหรือสตริง ซึ่งข้อมูลที่เป็นตัวเลขจะถูกแปลงเป็นตัวเลขทศนิยม ขณะที่ข้อมูลที่เป็นสตริงจะถูกแปลงด้วย StringIndexer และจะทำการ one-hot encoding เมื่อจำเป็น

**Example**

In [ ]:
from pyspark.ml.feature import RFormula

df = spark.createDataFrame(
    [(7, "US", 18, 1.0),
     (8, "CA", 12, 0.0),
     (9, "NZ", 15, 0.0)],
    ["id", "country", "hour", "clicked"])

**3. ChiSqSelector**

ChiSqSelector คือ เครื่องมือในการเลือกฟีเจอร์ที่ใช้การทดสอบ Chi-Squared (Chi²) เพื่อเลือกฟีเจอร์ที่มีความสัมพันธ์สูงกับตัวแปรเป้าหมายในข้อมูลที่มีลักษณะเป็นข้อมูลหมวดหมู่ (categorical data) โดยวิธีการนี้จะเลือกฟีเจอร์ที่มีความสัมพันธ์มากที่สุดระหว่างฟีเจอร์และตัวแปรเป้าหมายผ่านการทดสอบ Chi-Squared ของการเป็นอิสระ (independence test)

ChiSqSelector รองรับวิธีการเลือกฟีเจอร์ 5 แบบ ดังนี้

1. numTopFeatures เลือกฟีเจอร์ที่ดีที่สุดตามผลการทดสอบ Chi-Squared โดยเลือกจำนวนฟีเจอร์สูงสุดที่กำหนด (ตัวเลือกนี้จะเลือกฟีเจอร์ที่มีความสามารถในการทำนายดีที่สุด).

2. percentile คล้ายกับ numTopFeatures แต่จะเลือกฟีเจอร์เป็นสัดส่วนจากฟีเจอร์ทั้งหมด แทนการเลือกจำนวนที่แน่นอน.

3. fpr (False Positive Rate) เลือกฟีเจอร์ที่มีค่า p-value ต่ำกว่าค่าระดับที่กำหนด ซึ่งจะช่วยควบคุมอัตราความผิดพลาดที่เกิดจากการเลือกฟีเจอร์ที่ไม่สัมพันธ์จริง (false positives).

4. fdr (False Discovery Rate) ใช้กระบวนการ Benjamini-Hochberg ในการเลือกฟีเจอร์ที่มีอัตราการค้นพบผิดพลาด (false discovery rate) ต่ำกว่าค่าที่กำหนด.

5. fwe (Family-Wise Error Rate) เลือกฟีเจอร์ที่มีค่า p-value ต่ำกว่าค่าที่กำหนด ซึ่งจะถูกปรับขนาดตามจำนวนฟีเจอร์ทั้งหมด เพื่อควบคุมอัตราความผิดพลาดในระดับครอบครัว (family-wise error rate)

โดยปกติแล้ว ChiSqSelector จะใช้วิธี numTopFeatures เป็นค่าเริ่มต้น ซึ่งจะเลือกฟีเจอร์ที่ดีที่สุด 50 ตัว ผู้ใช้สามารถเลือกวิธีการเลือกฟีเจอร์ที่ต้องการได้โดยใช้เมธอด setSelectorType

**Example**

In [ ]:
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([
    (7, Vectors.dense([0.0, 0.0, 18.0, 1.0]), 1.0,),
    (8, Vectors.dense([0.0, 1.0, 12.0, 0.0]), 0.0,),
    (9, Vectors.dense([1.0, 0.0, 15.0, 0.1]), 0.0,)], ["id", "features", "clicked"])


**4. UnivariateFeatureSelector**

UnivariateFeatureSelector เป็นเครื่องมือที่ช่วยเลือกฟีเจอร์โดยพิจารณาความสัมพันธ์ระหว่างฟีเจอร์และเป้าหมาย และสามารถเลือกฟีเจอร์ได้โดยใช้หลายวิธีการตามโหมดที่กำหนด เช่น เลือกฟีเจอร์ที่ดีที่สุด, เลือกฟีเจอร์ตามอัตราความผิดพลาด หรือเลือกฟีเจอร์ที่มีค่า p-value ต่ำกว่าค่าที่กำหนด

โหมดการเลือกฟีเจอร์ (Selection Modes)

1. numTopFeatures เลือกฟีเจอร์จำนวนคงที่ที่ดีที่สุด
percentile: เลือกฟีเจอร์ตามสัดส่วนของฟีเจอร์ทั้งหมด (ไม่ใช่จำนวนที่แน่นอน)
2. fpr (False Positive Rate) เลือกฟีเจอร์ที่มีค่า p-value ต่ำกว่าค่าที่กำหนด เพื่อควบคุมอัตราการเลือกฟีเจอร์ที่เป็นบวกเท็จ
3. fdr (False Discovery Rate) ใช้กระบวนการ Benjamini-Hochberg เพื่อเลือกฟีเจอร์ที่มีอัตราความผิดพลาดต่ำกว่าค่าที่กำหนด
4. fwe (Family-Wise Error Rate) เลือกฟีเจอร์ที่มีค่า p-value ต่ำกว่าค่าที่กำหนด ซึ่งค่าของ threshold จะปรับตามจำนวนฟีเจอร์ทั้งหมด เพื่อควบคุมอัตราความผิดพลาดของการเลือกฟีเจอร์แบบครอบครัว

**Example**

In [72]:
from pyspark.ml.feature import UnivariateFeatureSelector
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([
    (1, Vectors.dense([1.7, 4.4, 7.6, 5.8, 9.6, 2.3]), 3.0,),
    (2, Vectors.dense([8.8, 7.3, 5.7, 7.3, 2.2, 4.1]), 2.0,),
    (3, Vectors.dense([1.2, 9.5, 2.5, 3.1, 8.7, 2.5]), 3.0,),
    (4, Vectors.dense([3.7, 9.2, 6.1, 4.1, 7.5, 3.8]), 2.0,),
    (5, Vectors.dense([8.9, 5.2, 7.8, 8.3, 5.2, 3.0]), 4.0,),
    (6, Vectors.dense([7.9, 8.5, 9.2, 4.0, 9.4, 2.1]), 4.0,)], ["id", "features", "label"])

selector = UnivariateFeatureSelector(featuresCol="features", outputCol="selectedFeatures",
                                     labelCol="label", selectionMode="numTopFeatures")
selector.setFeatureType("continuous").setLabelType("categorical").setSelectionThreshold(1)

UnivariateFeatureSelector_de9c08f3151d

In [73]:
result = selector.fit(df).transform(df)
result.show(truncate=False)

+---+-------------------------+-----+----------------+
|id |features                 |label|selectedFeatures|
+---+-------------------------+-----+----------------+
|1  |[1.7,4.4,7.6,5.8,9.6,2.3]|3.0  |[2.3]           |
|2  |[8.8,7.3,5.7,7.3,2.2,4.1]|2.0  |[4.1]           |
|3  |[1.2,9.5,2.5,3.1,8.7,2.5]|3.0  |[2.5]           |
|4  |[3.7,9.2,6.1,4.1,7.5,3.8]|2.0  |[3.8]           |
|5  |[8.9,5.2,7.8,8.3,5.2,3.0]|4.0  |[3.0]           |
|6  |[7.9,8.5,9.2,4.0,9.4,2.1]|4.0  |[2.1]           |
+---+-------------------------+-----+----------------+



**5. VarianceThresholdSelector**

VarianceThresholdSelector เป็นตัวเลือกที่ใช้ลบฟีเจอร์ที่มีความแปรปรวนต่ำ โดยจะลบฟีเจอร์ที่มีความแปรปรวน (variance) น้อยกว่าหรือเท่ากับค่า varianceThreshold ที่กำหนด หากไม่ตั้งค่า varianceThreshold ค่าเริ่มต้นจะเป็น 0 ซึ่งหมายความว่าเฉพาะฟีเจอร์ที่มีค่าแปรปรวนเป็น 0 (คือฟีเจอร์ที่มีค่าเหมือนกันทุกตัวอย่าง) จะถูกลบออก

**Example**

In [ ]:
from pyspark.ml.feature import VarianceThresholdSelector
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([
    (1, Vectors.dense([6.0, 7.0, 0.0, 7.0, 6.0, 0.0])),
    (2, Vectors.dense([0.0, 9.0, 6.0, 0.0, 5.0, 9.0])),
    (3, Vectors.dense([0.0, 9.0, 3.0, 0.0, 5.0, 5.0])),
    (4, Vectors.dense([0.0, 9.0, 8.0, 5.0, 6.0, 4.0])),
    (5, Vectors.dense([8.0, 9.0, 6.0, 5.0, 4.0, 4.0])),
    (6, Vectors.dense([8.0, 9.0, 6.0, 0.0, 0.0, 0.0]))], ["id", "features"])